In [ ]:
"""
Plotting simulation results

Authors: Milind Kumar Vaddiraju, ChatGPT, Copilot
"""

# Necessary imports
import copy
from datetime import datetime
import json
# %matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
import os
import pandas as pd
import pickle
import sys

sys.path.insert(0, os.path.abspath('..'))

from network_classes import *
from utils import *


In [ ]:
# Some common functions

def return_folder_paths(experiment_foldername):
    folder_names = []
    for folder in os.listdir(experiment_foldername):
        # Check if the folder is a directory
        if os.path.isdir(experiment_foldername + folder + "/"):
            folder_names.append(experiment_foldername + folder + "/")
            
    return folder_names

def obtain_plot_information(experiment_key, plotting_information, data_key):
        y = []
        x = []

        results_allUEs_per_lambda_contention = plotting_information[experiment_key]["results_allUEs_per_lambda_contention"]
        for lambda_value in results_allUEs_per_lambda_contention:
                x.append(lambda_value)
                y.append(\
                       results_allUEs_per_lambda_contention[lambda_value][data_key])
                
        y = [i for _, i in sorted(zip(x,y))]
        x = sorted(x)

        return (x, y)


def extract_plotting_data(folder_names, key_accessor):
    plotting_data_temp = {}
    for folder in folder_names:
        experiment_filename = folder + "experiment_parameters.pkl"
        with open(experiment_filename, "rb") as file:
            data = pickle.load(file)
        key = key_accessor[folder]
        # print(key)
        if key in plotting_data_temp:
            plotting_data_temp[key]["lambda_range"] = np.concatenate((plotting_data_temp[key]["lambda_range"], data["experiment_parameters"]["lambda_range"]))
            plotting_data_temp[key]["results_allUEs_per_lambda_contention"].update(data["results_allUEs_per_lambda_contention"])
            # assert plotting_data_temp[key]["schedule_contention"] == data["schedule_contention"] TODO: Create equality for the Schedule class
            assert plotting_data_temp[key]["percentile_to_plot"] == data["experiment_parameters"]["percentile_to_plot"]
        else:
            plotting_data_temp[key] = {
                "lambda_range": data["experiment_parameters"]["lambda_range"],
                "results_allUEs_per_lambda_contention": data["results_allUEs_per_lambda_contention"],
                "schedule_contention": data["schedule_contention"],
                "percentile_to_plot": data["experiment_parameters"]["percentile_to_plot"],
                "num_UEs": data["experiment_parameters"]["num_UEs"]
            }
    return plotting_data_temp


def write_csv(lambda_range, y_values, filename, path):
    # Ensure the inputs are numpy arrays or lists
    if isinstance(lambda_range, list):
        lambda_range = np.array(lambda_range)
    elif not isinstance(lambda_range, np.ndarray):
        raise ValueError("lambda_range must be a numpy array or list")

    if isinstance(y_values, list):
        y_values = np.array(y_values)
    elif not isinstance(y_values, np.ndarray):
        raise ValueError("y_values must be a numpy array or list")
    
    # Ensure the directory exists
    os.makedirs(path, exist_ok=True)
    
    # Create a DataFrame
    data = {
        "lambda_range": lambda_range,
        "y_values": y_values
    }
    df = pd.DataFrame(data)
    
    # Define the full file path
    file_path = os.path.join(path, f"{filename}.csv")
    
    # Write the DataFrame to a CSV file
    df.to_csv(file_path, index=False)




In [ ]:
# folder_names = [
#                 #"../results/simulation_4/2024_05_31_14_01_18/", 
#                 # "../results/simulation_4/2024_05_31_14_06_29/", 
#                 "../results/simulation_4/2024_05_31_14_10_05/",
#                 # "../results/simulation_4/10UEs_roundrobin_964B_80MHz/2024_05_19_19_08_11/", 
#                 "../results/simulation_4/2024_05_31_14_13_29/", 
#                 # "../results/simulation_4/2024_05_31_14_29_15/", 
#                 # "../results/simulation_4/2024_05_31_14_42_03/", 
#                 # "../results/simulation_4/2024_05_31_14_56_08/", 
#                 # "../results/simulation5/10UEs_dynamic_rr/2024_05_31_15_50_07/",
#                 "../results/simulation5/10UEs_dynamic_rr/2024_05_19_21_22_22/",
#                 # "../results/simulation6/2024_06_03_14_28_09/",
#                 # "../results/simulation6/2024_06_03_21_36_17/",
#                 # "../results/simulation6/2024_06_03_21_40_31/",
#                 # "../results/simulation6/2024_06_03_22_15_18/",
#                 "../results/simulation6/2024_06_03_22_26_24/",
#                 # "../results/simulation6/2024_06_03_22_33_29/",
#                 # "../results/simulation6/2024_06_04_12_47_09/",
#                 "../results/simulation6/2024_06_04_12_55_59/",
#                 # "../results/simulation6/2024_06_04_13_01_34/",
#                 # "../results/simulation6/2024_06_04_13_28_08/",
#                 "../results/simulation6/2024_06_04_13_39_22/",
#                 # "../results/simulation_4/2024_06_15_14_37_42/",
#                 # "../results/simulation_4/2024_06_14_16_27_04/",
#                 # "../results/simulation_4/2024_06_15_14_46_41/",
#                 # "../results/simulation_4/2024_06_15_15_19_31/",
#                 # "../results/simulation_4/2024_06_15_16_00_08/",
#                 # "../results/simulation_4/2024_06_15_19_20_28/",
#                 # "../results/simulation_4/2024_06_15_19_44_45/",
#                 # "../results/simulation_4/2024_06_16_22_34_24/",
#                 "../results/simulation_4/half_HVCs/2024_06_20_14_13_16/",
#                 "../results/simulation_4/half_HVCs/2024_06_20_15_27_47/"
#                 ]

# key_accessor_values = {
#     # "../results/simulation_4/2024_05_31_14_01_18/": "csma 0.1 per",
#     # "../results/simulation_4/2024_05_31_14_06_29/": "rr 1500 0.1 per",
#     "../results/simulation_4/2024_05_31_14_10_05/": "rr 1500 0 per",
#     # "../results/simulation_4/10UEs_roundrobin_964B_80MHz/2024_05_19_19_08_11/": "rr 1000 0 per",
#     "../results/simulation_4/2024_05_31_14_13_29/": "csma 0 per",
#     # "../results/simulation_4/2024_05_31_14_29_15/": "rr 800, 0 per",
#     # "../results/simulation_4/2024_05_31_14_42_03/": "rr 800, 0.1 per",
#     # "../results/simulation_4/2024_05_31_14_56_08/": "grr 2500, 0.1 per",
#     # "../results/simulation5/10UEs_dynamic_rr/2024_05_31_15_50_07/" : "drr cwmax",
#     "../results/simulation5/10UEs_dynamic_rr/2024_05_19_21_22_22/" : "drr cwmin",
#     # "../results/simulation6/2024_06_03_14_28_09/" : "max weight 1000",
#     # "../results/simulation6/2024_06_03_21_36_17/" : "oldest first 1000",
#     # "../results/simulation6/2024_06_03_21_40_31/" : "minimum latency 1000",
#     # "../results/simulation6/2024_06_03_22_15_18/" : "max weight dynamic",
#     "../results/simulation6/2024_06_03_22_26_24/" : "oldest first dynamic",
#     # "../results/simulation6/2024_06_03_22_33_29/" : "minimum latency dynamic",
#     # "../results/simulation6/2024_06_04_12_47_09/" : "max weight 1000, sbr 505.4",
#     "../results/simulation6/2024_06_04_12_55_59/" : "oldest first 1000, sbr 505.4",
#     # "../results/simulation6/2024_06_04_13_01_34/" : "minimum latency 1000, sbr 505.4",
#     # "../results/simulation6/2024_06_04_13_28_08/" : "max weight 1000, sbr 505.4, 3 period",
#     "../results/simulation6/2024_06_04_13_39_22/" : "oldest first 1000, sbr 505.4, 3 period",
#     # "../results/simulation_4/2024_06_15_14_37_42/" : "OFDMA slots, window 900",
#     # "../results/simulation_4/2024_06_14_16_27_04/" : "OFDMA slots, window 1500",
#     # "../results/simulation_4/2024_06_15_14_46_41/" : "OFDMA slots, window 1200",
#     # "../results/simulation_4/2024_06_15_15_19_31/" : "OFDMA slots, window 1700",
#     # "../results/simulation_4/2024_06_15_16_00_08/" : "OFDMA slots, window 2300",
#     # "../results/simulation_4/2024_06_15_19_20_28/" : "OFDMA slots, window 3400",
#     # "../results/simulation_4/2024_06_15_19_44_45/" : "OFDMA slots, window 8700"
#     # "../results/simulation_4/2024_06_16_22_34_24/" : "OFDMA slots, window 1500, 0.3 per",
#     "../results/simulation_4/half_HVCs/2024_06_20_14_13_16/" : "URLLC queue, MCS 2, wind = 1500",
#     "../results/simulation_4/half_HVCs/2024_06_20_15_27_47/" : "eMBB queue, MCS 6, 0 PER, wind = 1500"
# }

In [ ]:
# folder_names = []
# key_accessor_values = {}

# # CSMA with aggr 1
# folder_names.append("../results/simulation_4/2024_06_24_12_27_34/")
# key_accessor_values["../results/simulation_4/2024_06_24_12_27_34/"] = "CSMA, 0.1 PER, Aggr = 1, MCS 7"


# # CSMA with aggr 40
# folder_names.append("../results/hotnets_dummy_results/csma/2024_06_24_17_45_35/")
# key_accessor_values["../results/hotnets_dummy_results/csma/2024_06_24_17_45_35/"] = "CSMA, 0.1 PER, Aggr = 40, MCS 7"

# # # CSMA with aggr 100
# folder_names.append("../results/hotnets_dummy_results/csma/2024_06_24_17_40_23/")
# key_accessor_values["../results/hotnets_dummy_results/csma/2024_06_24_17_40_23/"] = "CSMA, 0.1 PER, aggr = 100, MCS 7"

# # # CSMA with aggr 150
# # folder_names.append("../results/hotnets_dummy_results/csma/2024_06_24_17_42_57/")
# # key_accessor_values["../results/hotnets_dummy_results/csma/2024_06_24_17_42_57/"] = "CSMA, 0.1 PER, aggr = 150, MCS 7"

# # ## rr 400
# folder_names.append("../results/hotnets_dummy_results/rr_round2/2024_06_24_19_08_05/")
# key_accessor_values["../results/hotnets_dummy_results/rr_round2/2024_06_24_19_08_05/"] = "RR, window size = 400, MCS 7, PER = 0.1"

# # # ## rr 600
# # folder_names.append("../results/hotnets_dummy_results/rr_round2/2024_06_24_19_17_04/")
# # key_accessor_values["../results/hotnets_dummy_results/rr_round2/2024_06_24_19_17_04/"] = "RR, window size = 600, MCS 7, PER = 0.1"

# # rr 1000
# folder_names.append("../results/hotnets_dummy_results/rr_round2/2024_06_24_18_42_53/")
# key_accessor_values["../results/hotnets_dummy_results/rr_round2/2024_06_24_18_42_53/"] = "RR, window size = 1000, MCS 7, PER = 0.1"

# # # rr 1500
# folder_names.append("../results/hotnets_dummy_results/rr_round2/2024_06_24_18_51_13/")
# key_accessor_values["../results/hotnets_dummy_results/rr_round2/2024_06_24_18_51_13/"] = "RR, window size = 1500, MCS 7, PER = 0.1"

# # rr 2000
# folder_names.append("../results/hotnets_dummy_results/rr_round2/2024_06_24_18_59_39/")
# key_accessor_values["../results/hotnets_dummy_results/rr_round2/2024_06_24_18_59_39/"] = "RR, window size = 2000, MCS 7, PER = 0.1"



# # folder_names.append("../results/hotnets_dummy_results/2024_06_24_14_40_00/")
# # key_accessor_values["../results/hotnets_dummy_results/2024_06_24_14_40_00/"] = "RR, window size = 400, MCS 7, PER = 0.1"


# # # Dynamic RR
# folder_names.append("../results/hotnets_dummy_results/dynamic_rr/2024_06_24_19_52_05/")
# key_accessor_values["../results/hotnets_dummy_results/dynamic_rr/2024_06_24_19_52_05/"] = "Dynamic RR, MCS 7, PER = 0.1"

# # # URLLC
# folder_names.append("../results/hotnets_dummy_results/half_hvcs/2024_06_24_22_18_54/")
# key_accessor_values["../results/hotnets_dummy_results/half_hvcs/2024_06_24_22_18_54/"] = "URLLC, MCS 2, PER = 0"

# # # eMBB
# folder_names.append("../results/hotnets_dummy_results/half_hvcs/2024_06_24_22_19_23/")
# key_accessor_values["../results/hotnets_dummy_results/half_hvcs/2024_06_24_22_19_23/"] = "eMBB, MCS 7, PER = 0.1"

# # # oldest first
# folder_names.append("../results/simulation6/2024_06_24_22_46_58/")
# key_accessor_values["../results/simulation6/2024_06_24_22_46_58/"] = "Dynamic oldest first, MCS 7, PER = 0.1"

In [ ]:
folder_names = []
key_accessor_values = {}


# CSMA with aggr 150
folder_names.append("../results/hotnets_legit_trial1/csma/2024_06_25_03_08_08/")
key_accessor_values["../results/hotnets_legit_trial1/csma/2024_06_25_03_08_08/"] = "CSMA_150"

# ## rr 400
# folder_names.append("../results/hotnets_legit_trial1/roundrobin/2024_06_25_17_12_11/")
# key_accessor_values["../results/hotnets_legit_trial1/roundrobin/2024_06_25_17_12_11/"] = "RR_400"


# # rr 2000
folder_names.append("../results/hotnets_legit_trial1/roundrobin/2024_06_25_12_36_28/")
key_accessor_values["../results/hotnets_legit_trial1/roundrobin/2024_06_25_12_36_28/"] = "RR_2000"



# # Dynamic RR
folder_names.append("../results/hotnets_legit_trial1/dynamic_rr/2024_06_25_17_01_25/")
key_accessor_values["../results/hotnets_legit_trial1/dynamic_rr/2024_06_25_17_01_25/"] = "Dynamic_rr"

# URLLC
folder_names.append("../results/hotnets_legit_trial1/half_hvcs/contention_based/2024_06_26_19_37_07/")
key_accessor_values["../results/hotnets_legit_trial1/half_hvcs/contention_based/2024_06_26_19_37_07/"] = "LLC"

# eMBB
folder_names.append("../results/hotnets_legit_trial1/half_hvcs/contention_based/2024_06_26_20_58_53/")
key_accessor_values["../results/hotnets_legit_trial1/half_hvcs/contention_based/2024_06_26_20_58_53/"] = "HBC"


# URLLC with dynamic HBC, URLLC slot fixed to 1500
# folder_names.append("../results/dynamic_HVCs/2024_10_07_20_15_32/")
# key_accessor_values["../results/dynamic_HVCs/2024_10_07_20_15_32/"] = "LLC, dynamic HBC slot, 1500 LLC slot, 0.95"

# # eMBB with dynamic HBC, URLLC slot fixed to 1500
# folder_names.append("../results/dynamic_HVCs/2024_10_07_19_57_31/")
# key_accessor_values["../results/dynamic_HVCs/2024_10_07_19_57_31/"] = "HBC, dynamic HBC slot, 1500 LLC slot, 0.95"

# URLLC with dynamic HBC, URLLC slot fixed to 1500, lambda_frac_embb = 1.1
# folder_names.append("../results/dynamic_HVCs/2024_10_07_20_37_31/")
# key_accessor_values["../results/dynamic_HVCs/2024_10_07_20_37_31/"] = "LLC, dynamic HBC slot, 1500 LLC slot, 1.1"

# # eMBB with dynamic HBC, URLLC slot fixed to 1500, lambda_frac_embb = 1.1
# folder_names.append("../results/dynamic_HVCs/2024_10_07_20_36_23/")
# key_accessor_values["../results/dynamic_HVCs/2024_10_07_20_36_23/"] = "HBC, dynamic HBC slot, 1500 LLC slot, 1.1"


# URLLC with dynamic HBC and LLC, lambda_frac_embb = 1.1, URLLC buffer = 200us
folder_names.append("../results/dynamic_HVCs/2024_10_07_23_11_05/")
key_accessor_values["../results/dynamic_HVCs/2024_10_07_23_11_05/"] = "LLC, dynamic HBC and LLC"

# URLLC with dynamic HBC and LLC, lambda_frac_embb = 1.1, URLLC buffer = 200us
folder_names.append("../results/dynamic_HVCs/2024_10_07_22_54_12/")
key_accessor_values["../results/dynamic_HVCs/2024_10_07_22_54_12/"] = "HBC, dynamic HBC and LLC"



# # URLLC
# folder_names.append("../results/OFDMA_experiments/pure_OFDMA/2024_10_05_16_22_56/")
# key_accessor_values["../results/OFDMA_experiments/pure_OFDMA/2024_10_05_16_22_56/"] = "LLC OFDMA UL + DL, 990 + 380"

# # UL OFDMA 8 slots
folder_names.append("../results/OFDMA_experiments/pure_OFDMA/2024_10_05_22_28_10/")
key_accessor_values["../results/OFDMA_experiments/pure_OFDMA/2024_10_05_22_28_10/"] = "Pure OFDMA UL, 8 slots, 1500"

# # UL OFDMA 8 slots
folder_names.append("../results/OFDMA_experiments/pure_OFDMA/2024_10_06_15_18_09/")
key_accessor_values["../results/OFDMA_experiments/pure_OFDMA/2024_10_06_15_18_09/"] = "Pure OFDMA UL, 8 slots, 2000"

# # UL OFDMA 8 slots
folder_names.append("../results/OFDMA_experiments/pure_OFDMA/2024_10_08_14_24_12/")
key_accessor_values["../results/OFDMA_experiments/pure_OFDMA/2024_10_08_14_24_12/"] = "Pure OFDMA UL, 8 slots, 2000, dynamic OFDMA"

# # UL OFDMA 8 slots
folder_names.append("../results/OFDMA_experiments/pure_OFDMA/2024_10_08_14_52_12/")
key_accessor_values["../results/OFDMA_experiments/pure_OFDMA/2024_10_08_14_52_12/"] = "Pure OFDMA UL, 8 slots, 1500, dynamic OFDMA"



# # UL OFDMA 8 slots
# folder_names.append("../results/OFDMA_experiments/OFDMA_HVCs/2024_10_07_10_10_59/")
# key_accessor_values["../results/OFDMA_experiments/OFDMA_HVCs/2024_10_07_10_10_59/"] = "UL OFDMA HBC, 8 slots, 1500"

# # DL OFDMA 1/9 slots
# folder_names.append("../results/OFDMA_experiments/OFDMA_HVCs/2024_10_07_10_20_29/")
# key_accessor_values["../results/OFDMA_experiments/OFDMA_HVCs/2024_10_07_10_20_29/"] = "DL CSMA HBC, 8 slots, 1500"

# URLLC
# folder_names.append("../results/hotnets_dummy_results/2024_06_26_16_58_10/")
# key_accessor_values["../results/hotnets_dummy_results/2024_06_26_16_58_10/"] = "LLC"

# # eMBB
# folder_names.append("../results/hotnets_dummy_results/2024_06_26_17_29_38/")
# key_accessor_values["../results/hotnets_dummy_results/2024_06_26_17_29_38/"] = "HBC"


# # # oldest first
# folder_names.append("../results/simulation6/hotnets_dummy/2024_06_26_12_24_49/")
# key_accessor_values["../results/simulation6/hotnets_dummy/2024_06_26_12_24_49/"] = "2000 oldest first, period 1"

# folder_names.append("../results/simulation6/hotnets_dummy/2024_06_26_12_43_04/")
# key_accessor_values["../results/simulation6/hotnets_dummy/2024_06_26_12_43_04/"] = "Dynamic oldest first, period 1"

# folder_names.append("../results/simulation6/hotnets_dummy/2024_06_26_12_55_54/")
# key_accessor_values["../results/simulation6/hotnets_dummy/2024_06_26_12_55_54/"] = "Dynamic oldest first, period 3"

# folder_names.append("../results/simulation6/hotnets_dummy/2024_06_26_13_11_55/")
# key_accessor_values["../results/simulation6/hotnets_dummy/2024_06_26_13_11_55/"] = "Dynamic oldest first, period 3, overhead 500"

# folder_names.append("../results/simulation6/hotnets_dummy/2024_06_26_13_33_47/")
# key_accessor_values["../results/simulation6/hotnets_dummy/2024_06_26_13_33_47/"] = "2000 min latency, period 1"

# folder_names.append("../results/simulation6/hotnets_dummy/2024_06_26_14_06_27/")
# key_accessor_values["../results/simulation6/hotnets_dummy/2024_06_26_14_06_27/"] = "2000 max weight, period 1"

# folder_names.append("../results/simulation6/hotnets_dummy/2024_06_26_15_15_14/")
# key_accessor_values["../results/simulation6/hotnets_dummy/2024_06_26_15_15_14/"] = "Dynamic oldest first, period 9"

# folder_names.append("../results/simulation6/hotnets_dummy/2024_06_26_15_32_22/")
# key_accessor_values["../results/simulation6/hotnets_dummy/2024_06_26_15_32_22/"] = "Dynamic oldest first, period 9, overhead = 500"


In [ ]:
label_prefix = {}
plotting_data = {}
plotting_keys = {}



plotting_data_temp = extract_plotting_data(folder_names, \
                                           key_accessor_values)


label_prefix["all"] = ""
plotting_data["all"] = plotting_data_temp



In [ ]:
plotting_keys["all"] = key_accessor_values.values()



save_file = False

scale = "linear"
percentile_to_plot = 99
percentile_filename = "percentile_latency_zoomed_" + scale + ".png"
percentile_slope_filename = "percentile_slope_zoomed_" + scale + ".png"
mean_filename = "mean_latency_zoomed_" + scale + ".png"
mean_slope_filename = "mean_slope_zoomed_" + scale + ".png"
n_packets_not_served_filename = "n_packets_not_served_zoomed_" + scale + ".png"
bus_occupancy_filename = "bus_occupancy_zoomed_" + scale + ".png"
n_wins_filename = "n_wins_zoomed_" + scale + ".png"
queue_slope_filename = "queue_slope_zoomed_" + scale + ".png"
# label_prefix_temp = "number of STAs = "

scaling_factor = 1e6


##### Generate color schemes

total_lines = sum(len(plotting_keys[schedule_key]) for schedule_key in label_prefix)

# Generate unique colors
colors = cm.get_cmap('tab20', total_lines)


##################### Plot the  percentile latency curve #####################
plot_linewidths = {}
plot_linewidths["dynamic_rr"] = 4
plot_linewidths["roundrobin_964B_80MHz"] = 4
line_index = 0

plt.figure(figsize=(10, 8))
for schedule_key in label_prefix:
        label_prefix_temp = label_prefix[schedule_key]
        if schedule_key in plot_linewidths:
                linewidth = plot_linewidths[schedule_key]
        else:
                linewidth = 2
        for plot_key in plotting_keys[schedule_key]:
                lambda_range, y_values = obtain_plot_information(plot_key, plotting_data[schedule_key], "percentile_latency")
                
                # write to file
                write_csv(lambda_range, y_values, f"{plot_key}_percentile_latency", "./csv_files")


                plt.plot(np.array(lambda_range)*scaling_factor, \
                        np.array(y_values)/1e3, ".-", label = label_prefix_temp + str(plot_key),\
                        linewidth=linewidth, color=colors(line_index))
                line_index += 1

plt.axhline(y=5, color='r', linestyle='--')

plt.xlabel("$\lambda$ (packets/s)", fontsize=15)
plt.ylabel("$99^{th}$ percentile latency (ms)", fontsize=15)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.legend(prop={'size': 10})
plt.grid()

if scale == "linear":
        plt.ylim(bottom=0)
        plt.ylim(top = 40)
        # plt.xlim(0,2000)
# plt.xlim(left=0)
# plt.ylim(0,50000)


if scale == "log":
        plt.yscale('log')
        plt.xscale('log')

title = ("$99^{th}$ percentile latency vs load\n" 
        )
plt.title(title, fontsize=18)
# Insert a textbox at the lowest y value of the plot and have y axis be the label

plt.tight_layout()
if save_file:
        plt.savefig(os.path.join(experiment_foldername, percentile_filename))
plt.show()



##################### Plot the  mean latency curve #####################
line_index = 0
plt.figure(figsize=(10, 8))


for schedule_key in label_prefix:
        label_prefix_temp = label_prefix[schedule_key]
        if schedule_key in plot_linewidths:
                linewidth = plot_linewidths[schedule_key]
        else:
                linewidth = 2
        for plot_key in plotting_keys[schedule_key]:
                lambda_range, y_values = obtain_plot_information(plot_key, plotting_data[schedule_key], "mean_latency")
                
                write_csv(lambda_range, y_values, f"{plot_key}_mean_latency", "./csv_files")
                
                
                plt.plot(np.array(lambda_range)*scaling_factor, \
                        np.array(y_values)/1e3, ".-", label = label_prefix_temp + str(plot_key),\
                        linewidth=linewidth, color=colors(line_index))
                line_index += 1

plt.xlabel("lambda (packets/s)")
plt.ylabel("Mean latency (ms)")
plt.legend(prop={'size': 10})
plt.grid()
plt.ylim(0,40)
# plt.xlim(0,1000)


if scale == "log":
        plt.yscale('log')

title = (f"Mean latency vs lambda,\n" 
        )
plt.title(title)
# Insert a textbox at the lowest y value of the plot and have y axis be the label

plt.tight_layout()
if save_file:
        plt.savefig(os.path.join(experiment_foldername, mean_filename))
plt.show()



##################### Plot the number of unserved packets #####################
line_index = 0
plt.figure(figsize=(10, 8))

for schedule_key in label_prefix:
        label_prefix_temp = label_prefix[schedule_key]
        for plot_key in plotting_keys[schedule_key]:
                lambda_range, y_values = obtain_plot_information(plot_key, plotting_data[schedule_key], "n_packets_not_served")
                plt.plot(np.array(lambda_range)*scaling_factor, \
                        np.array(y_values)/plotting_data[schedule_key][plot_key]["num_UEs"], ".-", label = label_prefix_temp + str(plot_key),\
                        linewidth=2, color=colors(line_index))
                line_index += 1
# plt.plot(n_packets_generated, percentiles)
plt.xlabel("lambda (packets/s)")
plt.ylabel("Unserved packets")
plt.legend(prop={'size': 10})
plt.grid()

if scale == "log":
        plt.yscale('log')

title = (f"Unserved packets vs lambda,\n" 
        )
plt.title(title)
# Insert a textbox at the lowest y value of the plot and have y axis be the label

plt.tight_layout()
if save_file:
        plt.savefig(os.path.join(experiment_foldername, n_packets_not_served_filename))
plt.show()